<h1>
<font color=Green> Assignment : Model Experimentation</font>
</h1>

<hr>

<h2>
<font color=Green>Import the Requisite Libraries:</font>
</h2>

In [5]:
# ignore the warnings in the output
import warnings
warnings.filterwarnings("ignore")

!pip install mlflow
!pip install pycaret
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 106.6 MB/s eta 0:00:00
   ━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 13.3 MB/s eta 0:00:00


In [1]:
# import libraries and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
from pycaret.classification import *

<hr>
<h2>
<font color=Green>Data Reading:</font>
</h2>

In [4]:
%%time

##Copy the cleaned data obtained from the data_cleaning notebook into the Data folder before proceeding
dataset = pd.read_csv('/Data/cleaned_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Data/cleaned_data.csv'

In [ ]:
dataset.head()

,created_date,city_tier,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,app_complete_flag,assistance_interaction,career_interaction,payment_interaction,social_interaction,syllabus_interaction
0,2021-07-01 01:38:17,3.0,Level3,Level0,Level0,2.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,2021-07-01 02:41:36,1.0,Level0,Level11,others,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,2021-07-01 03:25:23,1.0,Level3,Level0,others,3.0,0.0,1,0.0,0.0,0.0,0.0,0.0
3,2021-07-01 03:43:23,3.0,Level3,Level0,others,2.0,0.0,1,0.0,0.0,0.0,0.0,0.0
4,2021-07-01 03:43:29,1.0,Level3,Level0,others,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# drop the data column as it is not needed for training
dataset = dataset.drop(['created_date'], axis=1)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24977 entries, 0 to 24976
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_tier               24977 non-null  float64
 1   first_platform_c        24977 non-null  object 
 2   first_utm_medium_c      24977 non-null  object 
 3   first_utm_source_c      24977 non-null  object 
 4   total_leads_droppped    24977 non-null  float64
 5   referred_lead           24977 non-null  float64
 6   app_complete_flag       24977 non-null  int64  
 7   assistance_interaction  24977 non-null  float64
 8   career_interaction      24977 non-null  float64
 9   payment_interaction     24977 non-null  float64
 10  social_interaction      24977 non-null  float64
 11  syllabus_interaction    24977 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
# Convert the `city_tier` as categorical type variable, since it was categorical originally
dataset['city_tier'] = dataset.city_tier.astype('category')

<hr>
<h2>
<font color=Green>Set up Database: </font>
</h2>

In [ ]:
# create a connection and setup a SQLite database with the name "lead_scoring_model_experimentation.db" in
# 'Assignment/02_training_pipeline/notebook/' location
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection("lead_scoring_model_experimentation.db")

2.6.0


In [ ]:
# Once, your server is successfully running, create a mlflow tracking uri at "http://0.0.0.0:6006"
mlflow.set_tracking_uri("http://127.0.0.1:6006")

In [ ]:
mlflow.set_experiment('Lead_scoring_mlflow_experimentation')

<Experiment: artifact_location='mlflow-artifacts:/315449696685501283', creation_time=1740495214008, experiment_id='315449696685501283', last_update_time=1740495214008, lifecycle_stage='active', name='Lead_scoring_mlflow_experimentation', tags={}>

<hr>
<h2>
<font color=Green>Model Experimentation with pycaret: </font>
</h2>

In [ ]:
# setup pycaret
exp_lead_scoring = setup(data=dataset, target = 'app_complete_flag',
                         fold_shuffle=True,
                         session_id = 42,
                         normalize = True,
                         transformation = True,
                         remove_multicollinearity = True, multicollinearity_threshold = 0.95,
                         n_jobs=-1,use_gpu=True,
                         log_experiment=True,experiment_name='Lead_scoring_model_experimentation',
                         log_plots=True, log_data=True,
                         verbose=True,
                         log_profile=False)


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves t

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves t

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves t

,Description,Value
0,Session id,42
1,Target,app_complete_flag
2,Target type,Binary
3,Original data shape,"(24977, 12)"
4,Transformed data shape,"(24977, 44)"
5,Transformed train set shape,"(17483, 44)"
6,Transformed test set shape,"(7494, 44)"
7,Numeric features,7
8,Categorical features,4
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves t

AttributeError: 'ThreadLocalVariable' object has no attribute 'copy'

In [ ]:
# create a experimentation with pycaret and exclude ['gbc','knn','qda', 'dummy', 'svm', 'ada']
best_model = compare_models(fold = 10, exclude=['gbc','knn','qda', 'dummy', 'svm', 'ada'])

In [ ]:
# create a model which gives the highest accuracy
lightgbm = create_model('lightgbm', fold=10)

In [ ]:
# create feature importance plot
plot_model(lightgbm, plot='feature_all')

In [ ]:
# list of the features that needs to be there in the final encoded dataframe
ONE_HOT_ENCODED_FEATURES = ['total_leads_droppped', 'city_tier_3.0',
                            'city_tier_1.0','first_platform_c_Level0',
                            'city_tier_2.0', 'first_platform_c_others',
                            'first_utm_source_c_others', 'first_utm_medium_c_others',
                            'first_platform_c_Level3', 'referred_lead',
                            'first_platform_c_Level1', 'first_utm_medium_c_Level15',
                            'first_utm_medium_c_Level0', 'first_utm_medium_c_Level11',
                            'first_utm_source_c_Level0', 'first_utm_source_c_Level7',
                            'first_platform_c_Level2', 'first_utm_source_c_Level16',
                            'first_utm_medium_c_Level30', 'first_platform_c_Level7',
                            'first_platform_c_Level8', 'first_utm_medium_c_Level6',
                            'first_utm_medium_c_Level4', 'first_utm_medium_c_Level8',
                            'first_utm_source_c_Level2', 'first_utm_source_c_Level6',
                            'first_utm_medium_c_Level2', 'first_utm_source_c_Level4',
                            'first_utm_medium_c_Level9', 'first_utm_medium_c_Level3',
                            'first_utm_medium_c_Level26', 'app_complete_flag']

In [ ]:
FEATURES_TO_ENCODE = ['city_tier', 'first_platform_c', 'first_utm_medium_c', 'first_utm_source_c']

In [ ]:
df_encoded = pd.DataFrame(columns=ONE_HOT_ENCODED_FEATURES)
df_placeholder = pd.DataFrame()

In [ ]:
# encode the features using get_dummies()
for f in FEATURES_TO_ENCODE:
    if(f in dataset.columns):
        encoded = pd.get_dummies(dataset[f])
        encoded = encoded.add_prefix(f + '_')
        df_placeholder = pd.concat([df_placeholder, encoded], axis=1)
    else:
        print('Feature not found')

In [ ]:
# add the encoded features into a single dataframe
for feature in df_encoded.columns:
    if feature in dataset.columns:
        df_encoded[feature] = dataset[feature]
    if feature in df_placeholder.columns:
        df_encoded[feature] = df_placeholder[feature]
df_encoded.fillna(0, inplace=True)

In [ ]:
df_encoded.head()

In [ ]:
df_encoded.info()

<hr>
<h2>
<font color=Green>Model Experimentation after dropping features: </font>
</h2>

From the above feature tests we can claerly see that some of the features are not significant. We will now drop all the insignificant features and select only the significant ones.
The list of the significant features is
['total_leads_droppped', 'city_tier', 'referred_lead', 'first_platform_c', 'first_utm_medium_c', 'first_utm_source_c'].
So now you will train your model with onlly these features.

We will drop the following variables
['carrer_interaction'.'assistance_interaction', 'syllabus_interaction', 'social_interaction']

In [ ]:
# Setup Pycaret Enviornment with selected variables
from pycaret.classification import *

exp_lead_scoring = setup(data=df_encoded , target = 'app_complete_flag',
                   remove_multicollinearity = True, multicollinearity_threshold = 0.95,
                   fold_shuffle=True,
                   session_id = 42,
                   n_jobs=-1,use_gpu=True,
                   log_experiment=True,experiment_name='Lead_scoring_mlflow_experimentation',
                   log_plots=True, log_data=True,
                   silent=True, verbose=True,
                   log_profile=False)

In [ ]:
# create a experimentation with pycaret and exclude ['gbc','knn','qda', 'dummy', 'svm', 'ada'].
best_model = compare_models(fold = 10,exclude=['gbc','knn','qda', 'dummy', 'svm', 'ada'])

In [ ]:
# You should get lightgbm as the best performing model. So now we will train a lightGBM model manually using pycaret
lightgbm_fs = create_model('lightgbm', fold=10)

In [ ]:
#!pip install optuna

In [ ]:
# Tune the hyper parameters of the lightgbm model using optuna on 10 folds and optimise AUC as that was our system metric,
# hence we will optimise AUC
import optuna

tuned_lgbm_optuna,tuner_1 = tune_model(lightgbm_fs,
                                   search_library = 'optuna',
                                   fold = 5,
                                   optimize = 'auc',
                                   choose_better = True,
                                   return_tuner=True)

In [ ]:
# Print the final models configuration so that we can use it in the model retraining pipeline
print(tuned_lgbm_optuna)

In [ ]:
# Store the selected data columns DataFrame in the SQL Database
cnx = sqlite3.connect("lead_scoring_model_experimentation.db")

df_encoded.to_sql(name='model_experimentation', con=cnx,if_exists='replace', index=False)
cnx.close()